# COGS181 Final Project Report
William Kim A16657857<br>
COGS181 

Source Code:

## 1. Abstract
Convolutional Neural Networks have been the  result of decades of research and hard work in the field of artificial intelligence and computer vision. These sophisticated architectures have a wide range of applications such as image classficiaiton, object detection, and image segmentation. For this particular use case, I decided to explore the practicality of building my own CNN architecture and sheddding light on the steps I took to train my network on the CIFAR-10 dataset. 

CFIAR-10 comes with a pool of 32x32 rgb images that have decent quality due its low dimensionality. With 10 uniquely identifiable classes, it provided enough scope and flexibility to explore different variants of my network. Other sources like CIFAR-100 or Tiny ImageNet might have been a better choice for better feature extraction, but the priority here is not perfection. Thus our approach to image recognition in this project should be considered an investigative one. In this report, I attempt to train to and test my CNN with hyper-parameter tuning to maximize its performance.

## 2. Problem
The goal of my project was to create an architecture similar to the ones we created in the class assignments. For reference, these models came equipped with two convolutional layers and a two layer MLP. While not powerful by any means, they provided a stable starting point for this project's exploration. 

However, increasing the capability was the biggest focus of this task since we'll see how networks as simple as the one described above didn't perform well. Given this, it was clear that a lot of my work would have to be deal with changing certain aspects of the models. Did certain activation functions play a role? Or Is it intrinsically the fault of how I designed my layers? I kept these questions in mind everytime I trained and tested.

To keep things simple, I utilized accuracy scores as my metric for performance. 

## 3. Method
CIFAR-10 is a dataset that consists of 60 thousand 32x32 color images with 6K images per class. 
More details on this dataset can be found here https://www.cs.toronto.edu/~kriz/cifar.html


For the project, I utilized pytorch to iteratively build and expand on my network implementations. This was supplemented with hyperparameter tuning on activation functions and pooling methods in order to converge on an optimal model. As a benchmark, I later used ResNet18 to compare model accuracies. ResNet17  was originally trained on the ImageNet dataset which are much larger and complex.  To ensure consistency with the problem statement at hand, I made sure to train the network on the same CIFAR-10 dataset. Performance of ResNet18 came out to 75% 


## 4. Experiment

### Iteration 1
The first iteration comprises of only two convolutional layers designed to fit the 32x32 images. On top of ReLU, I added batch normalization to normalize activations in the layer - this rescales and centers data in order stabilize and accelerate training. To prevent overfitting, I included dropout with a rate of 20% - this technique  randomly sets a fraction of input units to zero during training. Lastly, I utilized CrossEntropyLoss and Stoachastic Gradient Descent for back propogation. After extracting the features, I pipelined the flattened input to a 2 layer MLP network for classification. This first iteration yielded an average accuracy of 64% which was quite poor.

| Layer          | Type            | Input Shape       | Output Shape    |
|:---------------|:----------------|:------------------|:----------------|
| Conv1          | Conv2d          | 3x32x32           | 10x32x32        |
| Activation1    | ReLU            | 10x32x32          | 10x32x32        |
| BatchNorm1     | BatchNorm2d     | 10x32x32          | 10x32x32        |
| MaxPool1       | MaxPool2d       | 10x32x32          | 10x16x16        |
| Dropout1       | Dropout         | 10x16x16          | 10x16x16        |
| Conv2          | Conv2d          | 10x16x16          | 20x16x16        |
| Activation2    | ReLU            | 20x16x16          | 20x16x16        |
| BatchNorm2     | BatchNorm2d     | 20x16x16          | 20x16x16        |
| MaxPool2       | MaxPool2d       | 20x16x16          | 20x8x8          |
| Dropout2       | Dropout         | 20x8x8            | 20x8x8          |
| Flatten        | Flatten         | 20x8x8            | 1280            |
| FC1            | Linear          | 1280              | 100             |
| Activation3    | ReLU            | 100               | 100             |
| FC2            | Linear          | 100               | 10              |


```
# For comparison, let's run the old network to see how well the new network performs
net_old = NetOld(F.relu)   # Create the network1 instance.
net_old.to(device)         # Move the network parameters to the specified device.
net_old.apply(init_weights)
optim_old = optim.SGD(net_old.parameters(),lr = 0.001,  momentum = 0.9)
loss_old = nn.CrossEntropyLoss()

output_old = fit(trainloader, net_old, loss_old, optim_old, epochs)
```

```
Accuracy of the network on the 10000 test images: 64 %
Accuracy of plane : 69 %
Accuracy of   car : 71 %
Accuracy of  bird : 58 %
Accuracy of   cat : 30 %
Accuracy of  deer : 56 %
Accuracy of   dog : 64 %
Accuracy of  frog : 75 %
Accuracy of horse : 72 %
Accuracy of  ship : 76 %
Accuracy of truck : 74 %
```

As we can see, the model accuracy is fairly low. At this point in my project, I felt that mass improvements could be made. The old model was clearly struggling to identify classes like cats, deers, and dogs which have complex features. Perhaps the CNN had too few trainable weights to capture those details compared to simple ones like ships and cars. In order to increase the performance and complexity, I decided to make some modifications to the network. 

### Iteration 2
In the second iteration of the CNN model, significant improvements were observed by simply introducing an additional layer. This resulted in a notable increase of 9-10% in model accuracy, showcasing the impact of increased complexity on performance.

To further assess the effectiveness of activation functions, three different permutations were experimented with: ReLU, Sigmoid, and Tanh.

ReLU (Rectified Linear Unit): ReLU is a widely used activation function that replaces all negative input values with zero. It is computationally efficient and helps mitigate the vanishing gradient problem during training. ReLU has been shown to accelerate convergence and improve the training of deep neural networks. Its simplicity and effectiveness make it a popular choice in many CNN architectures.

Sigmoid: The sigmoid activation function squashes input values to the range [0, 1]. It is useful for binary classification tasks where the output needs to be interpreted as a probability. However, sigmoid functions suffer from the vanishing gradient problem, especially for inputs far from zero, which can slow down training in deep networks. Despite its drawbacks, sigmoid activation functions are still used in certain scenarios, particularly in the output layer of binary classifiers.

Tanh (Hyperbolic Tangent): Tanh is similar to the sigmoid function but squashes input values to the range [-1, 1]. Like sigmoid, tanh is useful for binary classification tasks and can handle negative input values better. However, tanh also suffers from the vanishing gradient problem, particularly for inputs far from zero. Despite this, tanh activation functions are preferred over sigmoid in some cases due to their ability to output both positive and negative values, making them more suitable for tasks where the output range is centered around zero.

| Layer   | Type        | Input Shape | Output Shape |
|---------|-------------|-------------|--------------|
| Conv1   | Conv2d      | 3x32x32     | 32x32x32     |
| Activation1 | Activation | 32x32x32  | 32x32x32     |
| BatchNorm1  | BatchNorm2d | 32x32x32 | 32x32x32     |
| MaxPool1    | MaxPool2d   | 32x32x32  | 32x16x16     |
| Dropout1    | Dropout     | 32x16x16  | 32x16x16     |
| Conv2       | Conv2d      | 32x16x16  | 64x16x16     |
| Activation2 | Activation  | 64x16x16  | 64x16x16     |
| BatchNorm2  | BatchNorm2d | 64x16x16  | 64x16x16     |
| MaxPool2    | MaxPool2d   | 64x16x16  | 64x8x8       |
| Dropout2    | Dropout     | 64x8x8    | 64x8x8       |
| Conv3       | Conv2d      | 64x8x8    | 128x8x8      |
| Activation3 | Activation  | 128x8x8   | 128x8x8      |
| BatchNorm3  | BatchNorm2d | 128x8x8   | 128x8x8      |
| MaxPool3    | MaxPool2d   | 128x8x8   | 128x4x4      |
| Dropout3    | Dropout     | 128x4x4   | 128x4x4      |
| Flatten     | Flatten     | 128x4x4   | 2048         |
| FC1         | Linear      | 2048       | 512          |
| Activation4 | Activation  | 512        | 512          |
| FC2         | Linear      | 512        | 10           |


### ReLu Variant 
```
net1 = Net(F.relu)   # Create the network1 instance.
net1.to(device)         # Move the network parameters to the specified device.
net1.apply(init_weights)
optim1 = optim.SGD(net1.parameters(),lr = 0.001,  momentum = 0.9)
loss1 = nn.CrossEntropyLoss()

Accuracy of the network on the 10000 test images: 74 %
Accuracy of plane : 80 %
Accuracy of   car : 76 %
Accuracy of  bird : 66 %
Accuracy of   cat : 46 %
Accuracy of  deer : 78 %
Accuracy of   dog : 67 %
Accuracy of  frog : 77 %
Accuracy of horse : 81 %
Accuracy of  ship : 78 %
Accuracy of truck : 84 %
```

### Sigmoid Variant 

```
net2 = Net(F.sigmoid)   # Create the network2 instance.
net2.to(device)         # Move the network parameters to the specified device.
net2.apply(init_weights)
optim2 = optim.SGD(net2.parameters(), lr=0.001)
loss2 = nn.CrossEntropyLoss()

Accuracy of the network on the 10000 test images: 41 %
Accuracy of plane : 34 %
Accuracy of   car : 63 %
Accuracy of  bird : 25 %
Accuracy of   cat : 18 %
Accuracy of  deer : 34 %
Accuracy of   dog : 46 %
Accuracy of  frog : 39 %
Accuracy of horse : 45 %
Accuracy of  ship : 64 %
Accuracy of truck : 47 %
```

### Tanh variant
```
net3 = Net(F.tanh)   # Create the network3 instance.
net3.to(device)         # Move the network parameters to the specified device.
net3.apply(init_weights)
optim3 = optim.SGD(net3.parameters(), lr=0.001)
loss3 = nn.CrossEntropyLoss()

Accuracy of the network on the 10000 test images: 66 %
Accuracy of plane : 72 %
Accuracy of   car : 76 %
Accuracy of  bird : 51 %
Accuracy of   cat : 47 %
Accuracy of  deer : 60 %
Accuracy of   dog : 52 %
Accuracy of  frog : 77 %
Accuracy of horse : 74 %
Accuracy of  ship : 80 %
Accuracy of truck : 72 %
```

### Comparing Activation Functions
I initially thought I'd see equally comparable results but I was in fact wrong. It turns out ReLU was the most performant out of the three, although the model still had trouble identifying mostly cats. Perhaps if CIFAR-10 had more cat images, we'd see a different trend. Despite this, the results confirmed to me that I'd have to stick with ReLU as my activation function for further hyperparameter tuning.

### Iteration 3
In the latest iteration of the Convolutional Neural Network (CNN), the complexity has been enhanced by introducing an additional layer and experimenting with various pooling functions. Three different pooling methods were tested: average pooling, fractional pooling, and adaptive pooling. After training and testing, the results were anti-climactic. Average and Fractional performed just as well as Max Pooling and with an accuracy of 74% and 73% respectively, however I got significantly worse results for Adaptive Pooling with an accuracy of 42%. 

#### Average Pooling: 
This method computes the average value of each patch of the input feature map. It divides the input into non-overlapping rectangular regions and computes the average value for each region. Average pooling is effective in downsampling the feature maps while preserving important information. However, it may blur the features to some extent, as it computes the average.

#### Fractional Pooling:
Unlike traditional pooling methods, fractional pooling allows for flexible downsampling ratios. It divides the input into regions of variable sizes and computes the maximum value within each region. This approach enables adaptive downsampling, allowing the network to learn more efficiently from different spatial resolutions. Fractional pooling is particularly useful when dealing with inputs of varying sizes or aspect ratios.

#### Adaptive Pooling: 
Adaptive pooling dynamically adjusts the size of the pooling region based on the input size. Instead of specifying the kernel size, as in traditional pooling methods, adaptive pooling allows the network to adaptively determine the pooling region's size. This enables the model to handle inputs of different dimensions without requiring manual adjustment of the pooling parameters. Adaptive pooling is beneficial for maintaining spatial information and improving the model's robustness to input size variations.

By trying these diverse pooling methods into the CNN architecture, the model should see some adaptability and flexibility in handling different types of data and input sizes.

| Layer   | Type             | Input Shape | Output Shape |
|---------|------------------|-------------|--------------|
| Conv1   | Conv2d           | 3x32x32     | 32x32x32     |
| Activation1 | Activation    | 32x32x32    | 32x32x32     |
| BatchNorm1  | BatchNorm2d   | 32x32x32    | 32x32x32     |
| Pool1    | MaxPool2d       | 32x32x32    | 32x16x16     |
| Dropout1 | Dropout          | 32x16x16    | 32x16x16     |
| Conv2    | Conv2d           | 32x16x16    | 64x16x16     |
| Activation2 | Activation     | 64x16x16    | 64x16x16     |
| BatchNorm2  | BatchNorm2d   | 64x16x16    | 64x16x16     |
| Pool2    | MaxPool2d       | 64x16x16    | 64x8x8       |
| Dropout2 | Dropout          | 64x8x8      | 64x8x8       |
| Conv3    | Conv2d           | 64x8x8      | 128x8x8      |
| Activation3 | Activation     | 128x8x8     | 128x8x8      |
| BatchNorm3  | BatchNorm2d   | 128x8x8     | 128x8x8      |
| Pool3    | MaxPool2d       | 128x8x8     | 128x4x4      |
| Dropout3 | Dropout          | 128x4x4     | 128x4x4      |
| Conv4    | Conv2d           | 128x4x4     | 256x4x4      |
| Activation4 | Activation     | 256x4x4     | 256x4x4      |
| BatchNorm4  | BatchNorm2d   | 256x4x4     | 256x4x4      |
| Pool4    | MaxPool2d       | 256x4x4     | 256x2x2      |
| Dropout4 | Dropout          | 256x2x2     | 256x2x2      |
| Flatten  | Flatten          | 256x2x2     | 1024         |
| FC1      | Linear           | 1024        | 512          |
| Activation5 | Activation     | 512         | 512          |
| FC2      | Linear           | 512         | 10           |


## ResNet18 Benchmark comparison
ResNet18 uses a residual network which incorporates "residual blocks" or skip connections, which increases training efficiency and mitigates vanishing gradients. ResNet18 is considered to be realtively shallow with just 18 total layers and is typically used for resource-limited scenarios. For the project, I trained this network on CIFAR-10 to get a baseline metric. This would help gauge how well my own CNN performed. Surprisingly, ResNet18 performed just as well as my best model.

```
model = resnet18(pretrained=False)  # Not using pre-trained weights
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # Change the last fully connected layer for 10 classes
model.to(device)

Accuracy of the network on the 10000 test images: 73 %
Accuracy of plane : 81 %
Accuracy of   car : 84 %
Accuracy of  bird : 56 %
Accuracy of   cat : 51 %
Accuracy of  deer : 66 %
Accuracy of   dog : 61 %
Accuracy of  frog : 87 %
Accuracy of horse : 86 %
Accuracy of  ship : 83 %
Accuracy of truck : 81 %
```

## 5. Conclusion
In this project, we explored extensive hyperparameter tuning and experimentation with a custom CNN architecture. Throughout the project, we've seen that model complexity is important to grow to a certain degree and that activation and pooling functions play an equally significant role in the performance of the network. Our models iteratively built on top of each other to maximize their performance, hitting an upper bound of around 74% which is a significant increase from the initial 64% we saw in our first CNN. 

Additionally, we also explored the ResNet-18 architecture, a shallow yet powerful convolutional neural network designed to address the challenges of vanishing gradients during training. Our experimentation involved training ResNet-18 on the CIFAR-10 dataset to establish a baseline performance metric. Surprisingly, despite its relatively shallow depth compared to to other models, ResNet-18 performed comparably to our own, demonstrating its effectiveness in image classification tasks. This underscores the robustness and efficiency of the ResNet architecture, particularly in resource-limited scenarios.

In conclusion, our findings highlight the effectiveness of ResNet-18 as an effective baseline model for image classification tasks. Furthermore, our exploration underscores the value of hyperparameter tuning and experimentation in optimizing CNN architectures to achieve optimal performance. With more time and resources, I would pursuit much deeper exploration into the interpretability of ResNet18 and try to improve my own architecture's back propogation techniques following similar techniques. Additionally, I would want to scale to complex datasets with higher dimensionality in order to capture more feature maps, which might address the pitfalls with classes like cats and birds as we saw above. 




### 6. References